## Minimal Ollama Showcase


Quick sanity checks plus two ways to call the same JSON-focused model.

### 0. Confirm the local server is alive

In [1]:
!curl http://localhost:11434/

Ollama is running

In [2]:
!pip install requests -q

In [3]:
import json
import requests

tags = requests.get('http://localhost:11434/api/tags', timeout=30)
tags.raise_for_status()
models = [m['name'] for m in tags.json().get('models', [])]
print('Available models:', ', '.join(models) or 'none')
if any(m == 'kodzero-shorts-json:latest' for m in models):
    print('Model ready.')
else:
    print("Run 'ollama create' for kodzero-shorts-json first.")

Available models: kodzero-shorts-json:latest, kodzero-shorts:latest, qwen2.5:7b-instruct-q4_K_M, gemma2:9b-instruct-q4_K_M, llama3.1:8b-instruct-q4_K_M, SpeakLeash/bielik-7b-instruct-v0.1-gguf:Q4_K_S, deepseek-r1:7b
Model ready.


### 1. Plain requests call

In [4]:
url = 'http://localhost:11434/api/generate'
payload = {
    'model': 'kodzero-shorts-json:latest',
    'prompt': 'This is a transcript about installing Docker on a server...',
    'stream': False,
    'format': 'json'
}
raw_data = requests.post(url, json=payload, timeout=560).json()
raw_data

{'model': 'kodzero-shorts-json:latest',
 'created_at': '2025-11-15T18:08:36.771002714Z',
 'response': '{\n  "tytul": "Docker na serwerze - jak zainstalować i skonfigurować",\n  "opis": "Oto jak łatwo zainstalować i skonfigurować Docker na swoim serwerze.\\n\\ndocker,\\nserviser,\\nkonteneryzacja,\\nlinux,\\nubuntu,\\ndebian,\\nraspberry pi,\\nserver,\\nkontenery,\\n\\n#shorts #viralshorts #docker #serwer #kontenery",\n  "tagi": "docker na serwerze, instalacja docker, konfiguracja docker, linux, ubuntu, debian, raspberry pi, server, kontenery",\n  "kiedy": "2/3",\n  "miniaturka": "server: datacenter; hero: administrator; anchors: terminal, docker logo; overlay: minimalne instrukcje; callouts: 0; styling: high contrast, rule of thirds"\n}',
 'done': True,
 'done_reason': 'stop',
 'context': [128006,
  9125,
  128007,
  1432,
  2675,
  527,
  279,
  15592,
  18328,
  369,
  279,
  330,
  42,
  347,
  18811,
  1,
  13674,
  5613,
  627,
  2675,
  690,
  5371,
  264,
  7257,
  36815,
  505,

In [5]:
final_data = json.loads(raw_data['response'])
final_data

{'tytul': 'Docker na serwerze - jak zainstalować i skonfigurować',
 'opis': 'Oto jak łatwo zainstalować i skonfigurować Docker na swoim serwerze.\n\ndocker,\nserviser,\nkonteneryzacja,\nlinux,\nubuntu,\ndebian,\nraspberry pi,\nserver,\nkontenery,\n\n#shorts #viralshorts #docker #serwer #kontenery',
 'tagi': 'docker na serwerze, instalacja docker, konfiguracja docker, linux, ubuntu, debian, raspberry pi, server, kontenery',
 'kiedy': '2/3',
 'miniaturka': 'server: datacenter; hero: administrator; anchors: terminal, docker logo; overlay: minimalne instrukcje; callouts: 0; styling: high contrast, rule of thirds'}

### 2. Same idea with the ollama helper

In [6]:
!pip install ollama -q

In [7]:
import ollama

lib_response = ollama.generate(
    model='kodzero-shorts-json:latest',
    prompt='This is a transcript about installing Docker on a server...'
)
json.loads(lib_response['response'])

{'tytul': 'Docker na serwerze - jak zainstalować i skonfigurować #shorts #viralshorts',
 'opis': 'Oto jak zainstalować i skonfigurować Docker na swoim serwerze.\n\ndocker,\nserviser,\nkonteneryzacja,\nimage,\ncontainer,\nrepozytorium,\nkonfiguracja,\nporty,\nserwer,\n\n#shorts #viralshorts #docker #serwer #konteneryzacja',
 'tagi': 'docker instalacja, docker konfiguracja, docker serwer, docker kontenery, docker image, docker container, docker repozytorium, docker porty, docker bez gpu, docker na linux, docker na ubuntu',
 'kiedy': '2/3',
 'miniaturka': 'server: datacenter; hero: sysadmin; anchors: terminal, docker logo; overlay: minimalistic background; callouts: none; styling: vertical portrait, rule of thirds, tight crop, high contrast'}

# BONUS

In [3]:
from ollama import chat

stream = chat(
    model='qwen2.5:7b-instruct-q4_K_M',
    messages=[{'role': 'user', 'content': 'Why is the sky blue? Answer in POLISH'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

Kolor nieba zielono-niebieski to rezultatem przenoszenia światła słońca przez atmosferę Ziemi. Przeciwko cząsteczkom w powietrzu, np. gazom i pyłowi, czerwone i żółte fale światła są odbijane silniej niż niebieskie lub بنفسowe. Dzięki temu większość niebieskiego i بنفسowego światła przenika do naszych oczu, co sprawia, że widzimy niebo jako niebieskie.

Kolor nieba zielono-niebieski to rezultatem przenoszenia światła słońca przez atmosferę Ziemi. Przeciwko cząsteczkom w powietrzu, np. gazom i pyłowi, czerwone i żółte fale światła są odbijane silniej niż niebieskie lub بنفسowe. Dzięki temu większość niebieskiego i بنفسowego światła przenika do naszych oczu, co sprawia, że widzimy niebo jako niebieskie.